In [98]:
import pandas as pd
import numpy as np
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.colors as co

In [99]:
headers = ["timestamp", "raw_value", "voltage_level", "battery_fraction", "battery_soc"]
data = pd.read_csv('prototype_data/battery.csv', names=headers)

data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.sort_values(by='timestamp')
# Timestamp is not always synced
data_filtered = data[data['timestamp'].dt.year >= 2024]

In [100]:
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=("Voltage Level Over Time", "Battery SOC Over Time"))
fig.add_trace(
    go.Scatter(x=data_filtered['timestamp'], y=data_filtered['voltage_level'], mode='lines', name='Voltage'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=data_filtered['timestamp'], y=data_filtered['battery_soc'], mode='lines', name='Charge'),
    row=2, col=1
)

fig.update_layout(height=700, width=1400, title_text="Battery Discharge")
fig.update_xaxes(title_text="Timestamp", row=1, col=1)
fig.update_yaxes(title_text="Voltage Level", row=1, col=1)
fig.update_xaxes(title_text="Timestamp", row=2, col=1)
fig.update_yaxes(title_text="Battery SOC", row=2, col=1)

fig.show()

In [101]:
discharge_cycles = []
cycle_data = []
in_discharge = False
start_time = None

for idx, row in data_filtered.iterrows():
    soc = row['battery_soc']
    if in_discharge:
        current_cycle.append(row)
        if soc >= 100: # If level returns to 100%, discard the data
            in_discharge = False
            start_time = None
            current_cycle = []
    if not in_discharge and soc >= 99:
        in_discharge = True
        start_time = row['timestamp']
        current_cycle = []
        current_cycle.append(row)
    elif in_discharge and soc < 3: # Write time is 3 minutes (180s), so leaving room for potential microcontroller blackout (<3.2V)
        end_time = row['timestamp']
        if start_time is not None:
            discharge_time = end_time - start_time
            discharge_cycles.append(discharge_time)
            cycle_data.append(pd.DataFrame(current_cycle))
        in_discharge = False
        start_time = None

discharge_df = pd.DataFrame(discharge_cycles, columns=['discharge_time'])
display(discharge_df)

discharge_time_avg = discharge_df['discharge_time'].mean()

overview_df = pd.DataFrame({'discharge_time_avg': [discharge_time_avg]})
display(overview_df)

,discharge_time
0,0 days 03:32:44
1,0 days 03:19:22
2,0 days 03:14:08
3,0 days 03:52:03
4,0 days 03:10:37


,discharge_time_avg
0,0 days 03:25:46.800000


In [102]:
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=("Voltage Level Over All Discharge Cycles", "Battery SOC Over All Discharge Cycles"))
cols = co.DEFAULT_PLOTLY_COLORS

for idx, cycle_df in enumerate(cycle_data):
    cycle_df['elapsed_time'] = (cycle_df['timestamp'] - cycle_df['timestamp'].iloc[0]).dt.total_seconds()
    fig.add_trace(
        go.Scatter(x=cycle_df['elapsed_time'], y=cycle_df['voltage_level'], mode='lines', line=dict(width=2, color=cols[idx]), name=f'Cycle {idx + 1}', legendgroup=f'Cycle{idx + 1}', showlegend=True),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=cycle_df['elapsed_time'], y=cycle_df['battery_soc'], mode='lines', line=dict(width=2, color=cols[idx]), name=f'Cycle {idx + 1}', legendgroup=f'Cycle{idx + 1}', showlegend=False),
        row=2, col=1
    )

fig.update_layout(height=700, width=1400, title_text="Battery Discharge Cycles")
fig.update_xaxes(title_text="Elapsed Time (seconds)", row=1, col=1)
fig.update_yaxes(title_text="Voltage Level", row=1, col=1)
fig.update_xaxes(title_text="Elapsed Time (seconds)", row=2, col=1)
fig.update_yaxes(title_text="Battery SOC", row=2, col=1)

fig.show()